# Advanced NumPy

## `numpy` internals

In [1]:
import numpy as np
np.random.seed(2374)

In [2]:
arr = np.random.randint(10, size=(8,8))

In [3]:
arr

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

Information about array elements:

In [4]:
arr.itemsize, arr.dtype

(8, dtype('int64'))

In [ ]:
arr

How to step through array memory? Using `strides` property:

In [5]:
arr.strides

(64, 8)

I. e. `arr[0, 1]` is 8 bytes away from `arr[0, 0]` (one step along axis `1`), while `arr[1, 0]` is 64 bytes away from `arr[0, 0]` (one step along axis `0`).

In [6]:
arr.strides[0] == arr.shape[1] * arr.itemsize

True

But what about views?

In [7]:
arr_view = arr[::2, 1:]

In [8]:
arr

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

In [9]:
arr_view

array([[7, 6, 8, 2, 9, 2, 0],
       [6, 9, 3, 3, 5, 1, 2],
       [3, 9, 5, 2, 4, 5, 2],
       [8, 7, 4, 7, 3, 8, 2]])

Information about underlying array structure:

In [14]:
arr.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [16]:
arr[::2, :].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [15]:
arr_view.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

Views always have base array:

In [17]:
arr_view.base

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

In [18]:
arr.base

In [19]:
arr_view.base is arr

True

`strides` are provided with respect to the **underlying data** (which is the same between original array `arr` and view array `arr_view`!):

In [20]:
arr_view.strides

(128, 8)

In [21]:
arr_view.shape

(4, 7)

Since view is not contiguous, this relation is not True anymore:

In [22]:
arr_view.strides[0] == arr_view.shape[1] * arr_view.itemsize

False

Also, view starts not from byte 0 of the data, but steps 8 bytes inside the data:

In [26]:
np.byte_bounds(arr)[0]

95570217356256

In [27]:
np.byte_bounds(arr_view)[0] - np.byte_bounds(arr)[0]

8

In [28]:
np.byte_bounds(arr_view)

(95570217356264, 95570217356704)

In [29]:
np.byte_bounds(arr_view)[1] - np.byte_bounds(arr)[1]

-64

In [30]:
arr

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

In [31]:
arr_view

array([[7, 6, 8, 2, 9, 2, 0],
       [6, 9, 3, 3, 5, 1, 2],
       [3, 9, 5, 2, 4, 5, 2],
       [8, 7, 4, 7, 3, 8, 2]])

In [32]:
arr_view.strides

(128, 8)

In [33]:
arr.T

array([[5, 3, 8, 0, 4, 8, 9, 0],
       [7, 1, 6, 7, 3, 4, 8, 3],
       [6, 5, 9, 5, 9, 8, 7, 6],
       [8, 8, 3, 7, 5, 7, 4, 6],
       [2, 7, 3, 1, 2, 8, 7, 4],
       [9, 5, 5, 2, 4, 9, 3, 0],
       [2, 3, 1, 8, 5, 2, 8, 9],
       [0, 1, 2, 1, 2, 2, 2, 4]])

In [34]:
arr.T.strides

(8, 64)

Transpose reports similar strides, is it a view?

In [35]:
arr_view.T.strides

(8, 128)

In [36]:
arr_view.T[::2, 1:].base is arr

True

### Creating views manually

In [37]:
arr

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [3, 1, 5, 8, 7, 5, 3, 1],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [0, 7, 5, 7, 1, 2, 8, 1],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [8, 4, 8, 7, 8, 9, 2, 2],
       [9, 8, 7, 4, 7, 3, 8, 2],
       [0, 3, 6, 6, 4, 0, 9, 4]])

Take memory, associated with `arr`:

In [38]:
arr.data

In [39]:
a = arr[:,:]
a.shape

(8, 8)

Create a new array, poiting to the same memory:

In [ ]:
np.ndarray(buffer=arr.data, shape=arr.shape, dtype=arr.dtype)

Create a new *view* pointing to the same memory:

In [40]:
np.ndarray(buffer=arr.data, shape=(4, 8), dtype=arr.dtype, strides=(128, 8))

array([[5, 7, 6, 8, 2, 9, 2, 0],
       [8, 6, 9, 3, 3, 5, 1, 2],
       [4, 3, 9, 5, 2, 4, 5, 2],
       [9, 8, 7, 4, 7, 3, 8, 2]])

In [41]:
np.ndarray(buffer=arr.data, shape=(4, 8), dtype=arr.dtype, strides=(128, 8)).base is arr

True

## Cache effects

In [44]:
large_arr = np.random.randint(100, size=(1000000,))

In [45]:
STEP = 4
larger_arr = np.random.randint(100, size=(1000000*STEP,),
                               dtype=np.int8)

In [46]:
larger_arr.shape, large_arr.shape

((4000000,), (1000000,))

In [47]:
%timeit -n 100 -r 3 large_arr.sum()

377 µs ± 36.8 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [48]:
%timeit -n 100 -r 3 larger_arr[::STEP].sum()

773 µs ± 35.1 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [49]:
del large_arr, larger_arr

In [50]:
large_arr = np.random.randint(100, size=(5, 10000000))

In [51]:
large_arr.nbytes // (1024*1024)

381

In [52]:
large_arr

array([[63, 98, 45, ..., 88, 56, 12],
       [90, 62,  5, ..., 37, 64, 20],
       [21, 89, 26, ..., 40, 55, 11],
       [55, 23, 76, ..., 73, 61,  7],
       [20, 52, 51, ..., 91, 47, 35]])

In [53]:
large_arr.T

array([[63, 90, 21, 55, 20],
       [98, 62, 89, 23, 52],
       [45,  5, 26, 76, 51],
       ...,
       [88, 37, 40, 73, 91],
       [56, 64, 55, 61, 47],
       [12, 20, 11,  7, 35]])

In [54]:
large_arr.T.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [55]:
large_arr.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [56]:
%timeit -n 50 -r 3 large_arr.sum(axis=1).sum(axis=0)

35.6 ms ± 1.68 ms per loop (mean ± std. dev. of 3 runs, 50 loops each)


In [57]:
%timeit -n 50 -r 3 large_arr.sum(axis=0).sum()

101 ms ± 4.98 ms per loop (mean ± std. dev. of 3 runs, 50 loops each)


In [58]:
%timeit -n 50 -r 3 large_arr.T.sum(axis=0)

34.2 ms ± 128 µs per loop (mean ± std. dev. of 3 runs, 50 loops each)


In [59]:
large_arr.T.base is large_arr

True

## Memory allocations in computations

How long does it take to create a copy?

In [60]:
%timeit -n 20 -r 3 large_arr.copy()

148 ms ± 2.34 ms per loop (mean ± std. dev. of 3 runs, 20 loops each)


Operations create new arrays as well:

In [61]:
%timeit -n 20 -r 3 large_arr + 1

131 ms ± 9.91 ms per loop (mean ± std. dev. of 3 runs, 20 loops each)


`np.add` and `+` do more or less the same:

In [62]:
%timeit -n 20 -r 3 np.add(large_arr, 1)

131 ms ± 8.92 ms per loop (mean ± std. dev. of 3 runs, 20 loops each)


But in-place operations are faster (no allocations):

In [63]:
%timeit -n 100 -r 3 np.add(large_arr, 1, out=large_arr)

40 ms ± 2.18 ms per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [64]:
large_arr

array([[363, 398, 345, ..., 388, 356, 312],
       [390, 362, 305, ..., 337, 364, 320],
       [321, 389, 326, ..., 340, 355, 311],
       [355, 323, 376, ..., 373, 361, 307],
       [320, 352, 351, ..., 391, 347, 335]])

In [65]:
np.add(large_arr, 1, out=large_arr)

array([[364, 399, 346, ..., 389, 357, 313],
       [391, 363, 306, ..., 338, 365, 321],
       [322, 390, 327, ..., 341, 356, 312],
       [356, 324, 377, ..., 374, 362, 308],
       [321, 353, 352, ..., 392, 348, 336]])

In [66]:
large_arr

array([[364, 399, 346, ..., 389, 357, 313],
       [391, 363, 306, ..., 338, 365, 321],
       [322, 390, 327, ..., 341, 356, 312],
       [356, 324, 377, ..., 374, 362, 308],
       [321, 353, 352, ..., 392, 348, 336]])

# Broadcasting

How can we operate on arrays of different shapes? Should we reshape them first to a common shape?

In [68]:
arr_2d = np.random.randint(10, size=(10, 3))
arr_1d_1 = np.random.randint(10, size=(3, ))
arr_1d_2 = np.random.randint(10, size=(10, ))

In [69]:
arr_2d

array([[7, 8, 8],
       [9, 3, 9],
       [1, 0, 9],
       [8, 0, 1],
       [7, 8, 7],
       [8, 1, 0],
       [5, 6, 8],
       [4, 8, 7],
       [5, 6, 0],
       [2, 5, 3]])

In [70]:
arr_1d_1

array([6, 4, 9])

In [71]:
arr_1d_2

array([3, 5, 9, 9, 7, 7, 4, 2, 7, 3])

In [72]:
arr_2d, arr_1d_1

(array([[7, 8, 8],
        [9, 3, 9],
        [1, 0, 9],
        [8, 0, 1],
        [7, 8, 7],
        [8, 1, 0],
        [5, 6, 8],
        [4, 8, 7],
        [5, 6, 0],
        [2, 5, 3]]),
 array([6, 4, 9]))

Can we add the two?

In [73]:
arr_2d + arr_1d_1

array([[13, 12, 17],
       [15,  7, 18],
       [ 7,  4, 18],
       [14,  4, 10],
       [13, 12, 16],
       [14,  5,  9],
       [11, 10, 17],
       [10, 12, 16],
       [11, 10,  9],
       [ 8,  9, 12]])

But what was really added to `arr_2d`?

In [74]:
(arr_2d + arr_1d_1) - arr_2d

array([[6, 4, 9],
       [6, 4, 9],
       [6, 4, 9],
       [6, 4, 9],
       [6, 4, 9],
       [6, 4, 9],
       [6, 4, 9],
       [6, 4, 9],
       [6, 4, 9],
       [6, 4, 9]])

Can we do the same with `arr_1d_2`?

In [75]:
arr_2d + arr_1d_2

ValueError: operands could not be broadcast together with shapes (10,3) (10,) 

We need to change `arr_1d_2` shape first:

In [78]:
arr_1d_2.reshape((10,1)).shape

(10, 1)

In [76]:
arr_2d + arr_1d_2.reshape((10,1))

array([[10, 11, 11],
       [14,  8, 14],
       [10,  9, 18],
       [17,  9, 10],
       [14, 15, 14],
       [15,  8,  7],
       [ 9, 10, 12],
       [ 6, 10,  9],
       [12, 13,  7],
       [ 5,  8,  6]])

Alternatively, we can do:

In [79]:
np.expand_dims(arr_1d_2, axis=1)

array([[3],
       [5],
       [9],
       [9],
       [7],
       [7],
       [4],
       [2],
       [7],
       [3]])

In [80]:
arr_2d + np.expand_dims(arr_1d_2, axis=1)

array([[10, 11, 11],
       [14,  8, 14],
       [10,  9, 18],
       [17,  9, 10],
       [14, 15, 14],
       [15,  8,  7],
       [ 9, 10, 12],
       [ 6, 10,  9],
       [12, 13,  7],
       [ 5,  8,  6]])

It seems `arr_1d_2` was "replicated" in the same way as `arr_1d_1` but along different axis:

In [81]:
(arr_2d + np.expand_dims(arr_1d_2, axis=1)) - arr_2d

array([[3, 3, 3],
       [5, 5, 5],
       [9, 9, 9],
       [9, 9, 9],
       [7, 7, 7],
       [7, 7, 7],
       [4, 4, 4],
       [2, 2, 2],
       [7, 7, 7],
       [3, 3, 3]])

To reveal the pattern, let's try a `3D` array:

In [82]:
arr_3d = np.random.randint(10, size=(7, 10, 3))

In [83]:
arr_1d_1.shape

(3,)

In [84]:
arr_3d

array([[[9, 5, 1],
        [2, 4, 2],
        [9, 1, 4],
        [6, 3, 1],
        [3, 7, 4],
        [5, 3, 5],
        [4, 7, 1],
        [5, 1, 7],
        [9, 7, 4],
        [0, 1, 5]],

       [[9, 4, 4],
        [5, 5, 6],
        [9, 9, 2],
        [9, 2, 9],
        [9, 5, 7],
        [8, 7, 8],
        [4, 2, 7],
        [2, 3, 8],
        [0, 0, 1],
        [4, 9, 6]],

       [[2, 8, 6],
        [4, 3, 0],
        [9, 9, 6],
        [1, 4, 3],
        [9, 4, 5],
        [3, 8, 5],
        [7, 2, 1],
        [2, 1, 6],
        [5, 4, 2],
        [7, 6, 2]],

       [[6, 7, 9],
        [8, 8, 4],
        [2, 0, 5],
        [1, 8, 9],
        [8, 3, 1],
        [7, 3, 4],
        [0, 0, 4],
        [7, 8, 1],
        [4, 9, 5],
        [2, 4, 5]],

       [[1, 7, 0],
        [8, 3, 9],
        [8, 8, 8],
        [8, 7, 6],
        [2, 3, 8],
        [5, 4, 1],
        [8, 1, 3],
        [2, 4, 8],
        [2, 3, 9],
        [2, 3, 8]],

       [[2, 2, 2],
        [1, 8, 4],
  

In [85]:
arr_3d + arr_1d_1

array([[[15,  9, 10],
        [ 8,  8, 11],
        [15,  5, 13],
        [12,  7, 10],
        [ 9, 11, 13],
        [11,  7, 14],
        [10, 11, 10],
        [11,  5, 16],
        [15, 11, 13],
        [ 6,  5, 14]],

       [[15,  8, 13],
        [11,  9, 15],
        [15, 13, 11],
        [15,  6, 18],
        [15,  9, 16],
        [14, 11, 17],
        [10,  6, 16],
        [ 8,  7, 17],
        [ 6,  4, 10],
        [10, 13, 15]],

       [[ 8, 12, 15],
        [10,  7,  9],
        [15, 13, 15],
        [ 7,  8, 12],
        [15,  8, 14],
        [ 9, 12, 14],
        [13,  6, 10],
        [ 8,  5, 15],
        [11,  8, 11],
        [13, 10, 11]],

       [[12, 11, 18],
        [14, 12, 13],
        [ 8,  4, 14],
        [ 7, 12, 18],
        [14,  7, 10],
        [13,  7, 13],
        [ 6,  4, 13],
        [13, 12, 10],
        [10, 13, 14],
        [ 8,  8, 14]],

       [[ 7, 11,  9],
        [14,  7, 18],
        [14, 12, 17],
        [14, 11, 15],
        [ 8,  7, 17],
  

In [86]:
(arr_3d + arr_1d_1) - arr_3d

array([[[6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9]],

       [[6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9]],

       [[6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9]],

       [[6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9]],

       [[6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9],
        [6, 4, 9]],

       [[6, 4, 9],
        [6, 4, 9],
  

In [87]:
arr_3d.shape, arr_1d_1.shape

((7, 10, 3), (3,))

Can we do the same with `arr_1d_2`?

In [88]:
arr_3d + arr_1d_2

ValueError: operands could not be broadcast together with shapes (7,10,3) (10,) 

In [89]:
arr_3d.shape, arr_1d_2.shape, np.expand_dims(arr_1d_2, axis=1).shape

((7, 10, 3), (10,), (10, 1))

In [90]:
(arr_3d + np.expand_dims(arr_1d_2, axis=1)) - arr_3d

array([[[3, 3, 3],
        [5, 5, 5],
        [9, 9, 9],
        [9, 9, 9],
        [7, 7, 7],
        [7, 7, 7],
        [4, 4, 4],
        [2, 2, 2],
        [7, 7, 7],
        [3, 3, 3]],

       [[3, 3, 3],
        [5, 5, 5],
        [9, 9, 9],
        [9, 9, 9],
        [7, 7, 7],
        [7, 7, 7],
        [4, 4, 4],
        [2, 2, 2],
        [7, 7, 7],
        [3, 3, 3]],

       [[3, 3, 3],
        [5, 5, 5],
        [9, 9, 9],
        [9, 9, 9],
        [7, 7, 7],
        [7, 7, 7],
        [4, 4, 4],
        [2, 2, 2],
        [7, 7, 7],
        [3, 3, 3]],

       [[3, 3, 3],
        [5, 5, 5],
        [9, 9, 9],
        [9, 9, 9],
        [7, 7, 7],
        [7, 7, 7],
        [4, 4, 4],
        [2, 2, 2],
        [7, 7, 7],
        [3, 3, 3]],

       [[3, 3, 3],
        [5, 5, 5],
        [9, 9, 9],
        [9, 9, 9],
        [7, 7, 7],
        [7, 7, 7],
        [4, 4, 4],
        [2, 2, 2],
        [7, 7, 7],
        [3, 3, 3]],

       [[3, 3, 3],
        [5, 5, 5],
  

Broadcasting rules:
    
- All input arrays with `ndim` smaller than the input array of largest `ndim`, have 1’s **prepended** to their shapes.
- The size in each dimension of the output shape is the **maximum** of all the input sizes in that dimension.
- An input can be used in the calculation if its size in a particular dimension either **matches** the output size in that dimension, or **is exactly 1**.
- If an input has a dimension of size 1 in its shape, the first data entry in that dimension will be used for all calculations along that dimension. In other words, the stepping machinery of a `ufunc` will simply not step along that dimension (stride will be 0 for that dimension).

### How broadcasting really works

What happens, when we add a unit dimension somewhere?

In [93]:
arr_1d_1[np.newaxis, :]

array([[6, 4, 9]])

In [107]:
np.expand_dims(arr_1d_1, axis=0)

array([[6, 4, 9]])

In [110]:
arr_1d_1.reshape(1,-1)

array([[6, 4, 9]])

In [94]:
arr_1d_1[np.newaxis, :].strides

(0, 8)

`strides[0]` is `0`, which means we can use dimension `0` of `arr_1d_1[np.newaxis, :]` in any (underlying, C) loop with any number of iterations. Let's emulate this in pure Python:

In [95]:
arr_1d_1_bc = arr_1d_1[np.newaxis, :]
arr_1d_1_bc

array([[6, 4, 9]])

In [96]:
arr_2d

array([[7, 8, 8],
       [9, 3, 9],
       [1, 0, 9],
       [8, 0, 1],
       [7, 8, 7],
       [8, 1, 0],
       [5, 6, 8],
       [4, 8, 7],
       [5, 6, 0],
       [2, 5, 3]])

In [97]:
for i in range(arr_2d.shape[0]):

    print(f"Adding elements of row {i}")

    for j in range(arr_2d.shape[1]):
        arr_2d_address = arr_2d.strides[1] * j + arr_2d.strides[0] * i
        arr_1d_address = arr_1d_1_bc.strides[1] * j + arr_1d_1_bc.strides[0] * i

        print(f"\tarr_2d address: {arr_2d_address}")
        print(f"\tarr_1d_1_bc address: {arr_1d_address}")
    print("-" * 80)

Adding elements of row 0
	arr_2d address: 0
	arr_1d_1_bc address: 0
	arr_2d address: 8
	arr_1d_1_bc address: 8
	arr_2d address: 16
	arr_1d_1_bc address: 16
--------------------------------------------------------------------------------
Adding elements of row 1
	arr_2d address: 24
	arr_1d_1_bc address: 0
	arr_2d address: 32
	arr_1d_1_bc address: 8
	arr_2d address: 40
	arr_1d_1_bc address: 16
--------------------------------------------------------------------------------
Adding elements of row 2
	arr_2d address: 48
	arr_1d_1_bc address: 0
	arr_2d address: 56
	arr_1d_1_bc address: 8
	arr_2d address: 64
	arr_1d_1_bc address: 16
--------------------------------------------------------------------------------
Adding elements of row 3
	arr_2d address: 72
	arr_1d_1_bc address: 0
	arr_2d address: 80
	arr_1d_1_bc address: 8
	arr_2d address: 88
	arr_1d_1_bc address: 16
--------------------------------------------------------------------------------
Adding elements of row 4
	arr_2d address: 96
	

In [99]:
np.arange(10) + np.arange(10).reshape(10,1)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
       [ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [ 6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       [ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
       [ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]])

In [103]:
(np.arange(10) + np.arange(10).reshape(10,1)) - np.arange(10)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       [6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
       [7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
       [8, 8, 8, 8, 8, 8, 8, 8, 8, 8],
       [9, 9, 9, 9, 9, 9, 9, 9, 9, 9]])